In [494]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm, tqdm_pandas
from geopy.distance import great_circle
import json
from shapely.geometry import shape, Point, Polygon
from rtree import index

### 1. Reverse geocode latitude and longitude to zip code & ZCTA.

Load zip codes and subset NYC zips.

In [408]:
# alternate zips; not very precise since they overlap a lot...
zips_alt = pd.read_csv('data/zipcode-database.csv')
zips_alt = zips_alt[zips_alt.State == 'NY']

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
zips = pd.read_csv('data/zipcode/zipcode.csv')

In [4]:
# should find better way of doing this that isn't just me hand picking the indexes
zips = zips[zips.state == 'NY']
man_si_bx = zips.loc[3343:3646, :]
queens_bk = zips.loc[3845:3971, :]
rockaways = zips.loc[4030:4037, :]
nyc_zips = pd.concat([man_si_bx, queens_bk, rockaways])

In [55]:
# make a list of data
zip_list = zip(zip(nyc_zips.latitude.values, nyc_zips.longitude.values), nyc_zips.zip.values)

Define zip matching functions using spherical distance.

In [40]:
def get_distance(zip_pair, point_pair):
    """Get distance between two (lat, lon) pairs."""
    return great_circle(zip_pair, point_pair).miles

In [189]:
def get_zip(point, zip_list):
    """Get matching zip code given a (lat, lon) pair. Based on distance."""
    # point = (lat, lon)
    distances = [(get_distance(point, p[0]), p[1]) for p in zip_list]
    return min(distances)[1]

In [660]:
def add_zips_df(df, lat, lon):
    """Add zip codes to df with (lat, lon) given df and names of lat, lon columns. 
    Implemented with tqdm to show progress bar.
    """
    df_pairs = zip(df[lat], df[lon])
    zip_results = [get_zip(coord, zip_list) for coord in tqdm(df_pairs)]
    df.loc[:, 'zipcode'] = zip_results

Define zip matching functions using straight difference in (lat, lon) numbers.

In [183]:
def get_diff(zip_pair, point_pair):
    return sum((abs(zip_pair[0] - point_pair[0]), abs(zip_pair[1] - point_pair[1])))

In [190]:
def get_zip_diff(point, zip_list):
    distances = [(get_diff(point, p[0]), p[1]) for p in zip_list]
    return min(distances)[1]

In [187]:
def add_zips_df_diff(df, lat, lon):
    """Add zip codes to df with (lat, lon) given df and names of lat, lon columns. 
    Implemented with tqdm to show progress bar.
    """
    df_pairs = zip(df[lat], df[lon])
    zip_results = [get_zip_diff(coord, zip_list) for coord in tqdm(df_pairs)]
    df.loc[:, 'zipcode_diff'] = zip_results

Add zips to original perceived safety data.

In [651]:
ss_orig = pd.read_json('data/streetscore_original.json')

In [695]:
def clean_ss_df(df):
    """Clean original ss data df."""
    nyc = df[df.City == 'New York City']
    nyc = df[df['QS Safer'] != '#VALUE!']
    keep = ['Lat', 'Lon', 'QS Safer', 'QS Unique', 'QS Upperclass', 'Error in QS Safer', 'Error in QS Unique', 'Error in QS Upperclass']
    return nyc[keep]

In [696]:
ss_orig_nyc = clean_ss_df(ss_orig)

In [697]:
add_zips_df(ss_orig_nyc, 'Lat', 'Lon')

Add zips to predicted perceived safety data.

In [130]:
ss = pd.read_csv('data/streetscore_newyorkcity.csv')

In [156]:
add_zips_df(ss, 'latitude', 'longitude')

Add zips to crime data.

In [158]:
crime = pd.read_csv('data/NYPD_7_Major_Felony_Incidents.csv')

In [159]:
def get_lat(x):
    """Get latitude given tuple pair of (latitude, longitude)."""
    try:
        return x.translate(None, "'(),").split()[0]
    except:
        return x
    
def get_lon(x):
    """Get longitude given tuple pair of (latitude, longitude)."""
    try:
        return x.translate(None, "'(),").split()[1]
    except:
        return x

In [160]:
crime['lat'] = crime['Location 1'].apply(get_lat)
crime['lon'] = crime['Location 1'].apply(get_lon)

In [162]:
add_zips_df(crime, 'lat', 'lon')

### 2. Match zip codes to Census ZCTAs

In [286]:
zcta = pd.read_csv('data/zip_to_zcta_2015.csv')

In [300]:
zcta_dict = dict(zip(zcta.ZIP.values, zcta.ZCTA.values))

In [402]:
def add_zcta_df(df, zipcol):
    df['zcta'] = df[zipcol].apply(lambda x: zcta_dict[x] if x in zcta_dict else '')

In [ ]:
add_zcta_df(ss, 'zipcode')
add_zcta_df(crime, 'zipcode')

In [698]:
add_zcta_df(ss_orig_nyc, 'zipcode')

In [405]:
crime2011 = crime[crime['Occurrence Year'] == 2011]
crime2014 = crime[crime['Occurrence Year'] == 2014]

### 3. Get Census block group by latitude and longitude.

In [483]:
# load geojson file with shapes - state census block groups 
with open('data/tl_2010_36_bg10/blockgroups.json', 'r') as f:
    js_bg = json.load(f)

In [435]:
# nyc census blocks from nyc open data
with open('data/nyc2010bg/nyc2010cb.json', 'r') as f:
    js_cb = json.load(f)
    
# nyc census tracks from nyc open data
with open('data/nyc2010ct/nyc2010ct.json', 'r') as f:
    js_ct = json.load(f)

In [500]:
def get_fips(coord, json):
    """Return fips code given (lat, lon) and geojson file with geographies.
    No rtree indexing.
    """
    point = Point(coord[1], coord[0])
    for feature in json['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return feature['properties']

In [501]:
# def get_fips_index(coord, json):
#     """Return fips code given (lat, lon) and geojson file. Uses rtree indexing to find intersections before
#     checking if a shape contains the point.
#     """
#     # create list of polygons and properties
#     polygons = [shape(feature['geometry']) for feature in json['features']]
#     properties = [feature['properties'] for feature in json['features']]
    
#     # create r-tree index
#     idx = index.Index()
#     for pos, poly in enumerate(polygons):
#         idx.insert(pos, poly.bounds)

#     # search for point in index
#     point = Point(coord[1], coord[0])
#     poly_idx = [i for i in idx.intersection((point.coords[0])) if point.within(polygons[i])]
#     for num, idx in enumerate(poly_idx, 1):
#         return properties[idx]

In [523]:
def add_fips_df(df, lat, lon, json):
    """Add fips data to df with (lat, lon) given df and names of lat, lon columns. 
    Implemented with tqdm to show progress bar.
    """
    df_pairs = zip(df[lat], df[lon])
    zip_results = [get_fips(coord, json) for coord in tqdm(df_pairs)]
    df.loc[:, 'fips_data'] = zip_results

In [634]:
def add_fips_idx_df(df, lat, lon, json):
    """Add fips codes to df. Use rtree indexing to find intersections before
    checking if a shape contains the point."""
    # create list of polygons and properties
    polygons = [shape(feature['geometry']) for feature in json['features']]
    properties = [feature['properties'] for feature in json['features']]
    
    # create r-tree index
    idx = index.Index()
    for pos, poly in enumerate(polygons):
        idx.insert(pos, poly.bounds)

    # get points to search
    df_pairs = zip(df[lat], df[lon])
    
    # search for point in index
    results = []
    for coord in tqdm(df_pairs):
        point = Point(float(coord[1]), float(coord[0]))
        try:
            poly_idx = [i for i in idx.intersection((point.coords[0])) if point.within(polygons[i])]
            if len(poly_idx) > 0:
                for num, idx_result in enumerate(poly_idx, 1):
                    results.append(properties[idx_result])
            else:
                results.append('no result')
        except:
            results.append('error')
            continue

    # get block groups
    block_groups = []
    for result in results:
        try:
            block_groups.append(result['GEOID10'])
        except:
            block_groups.append('no result')
    
    # get tracts
    tracts = []
    for group in block_groups:
        try:
            tracts.append(group[:-1])
        except:
            tracts.append('no result')
    
    # update df with results
    df.loc[:, 'fips_detail'] = results
    df.loc[:, 'fips_bg'] = block_groups
    df.loc[:, 'fips_tract'] = tracts

In [612]:
%time add_fips_idx_df(ss, 'latitude', 'longitude', js_bg)

CPU times: user 2min 45s, sys: 1.65 s, total: 2min 47s
Wall time: 2min 51s


In [635]:
%time add_fips_idx_df(crime, 'lat', 'lon', js_bg)

CPU times: user 8min 42s, sys: 6.37 s, total: 8min 48s
Wall time: 8min 55s


In [699]:
add_fips_idx_df(ss_orig_nyc, 'Lat', 'Lon', js_bg)

In [667]:
print ss_orig_nyc[ss_orig_nyc.fips_bg == 'no result'].shape[0]
print ss[ss.fips_bg == 'no result'].shape[0]
print crime[crime.fips_bg == 'no result'].shape[0]

0
13
71


#### Pickle

In [700]:
with open('data/pickled/ss_orig_nyc.pkl', 'w') as picklefile:
    pickle.dump(ss_orig_nyc, picklefile)

In [671]:
with open('data/pickled/ss_withzip.pkl', 'w') as picklefile:
    pickle.dump(ss, picklefile)

In [672]:
with open('data/pickled/crime_withzip.pkl', 'w') as picklefile:
    pickle.dump(crime, picklefile)

### Compare actual crime vs. perceived safety scores

#### Perceived safety scores

Normalize q-scores to 0-10 range.

In [689]:
def normalize(df, score_col, norm_col, range_start = 0, range_end = 10):
    min_score = float(df[score_col].min())
    max_score = float(df[score_col].max())
    norm_func = lambda score: range_start + (float(score) - min_score) * (range_end - range_start) / (max_score - min_score)
    normalized = [norm_func(score) for score in df[score_col].tolist()]
    df.loc[:, norm_col] = normalized

In [701]:
normalize(ss, 'q-score', 'q_norm')
normalize(ss_orig_nyc, 'QS Safer', 'q_norm')

Convert/invert q-score of safety to danger score.

In [730]:
normalize(ss, 'q-score', 'q_norm_rev', range_start = 10, range_end = 0)
normalize(ss_orig_nyc, 'QS Safer', 'q_norm_rev', range_start = 10, range_end = 0)

Get average q-score by zip code.

In [644]:
def agg_qscores(df, geo_col, score_col):
    grouped = df.groupby(geo_col)
    scores = pd.DataFrame(grouped[score_col].mean())
    return scores

In [703]:
ssbyzcta = agg_qscores(ss, 'zcta', 'q_norm')
ss_orig_byzcta = agg_qscores(ss_orig_nyc, 'zcta', 'q_norm')

In [735]:
ssbyz_danger = agg_qscores(ss, 'zcta', 'q_norm_rev')
ssorigbyz_danger = agg_qscores(ss_orig_nyc, 'zcta', 'q_norm_rev')

Get average q-score by Census tract.

In [709]:
ss_by_ct = agg_qscores(ss, 'fips_tract', 'q_norm')
ssorig_by_ct = agg_qscores(ss_orig_nyc, 'fips_tract', 'q_norm')

Export for cartodb viz

In [706]:
def zip_to_cartodb(df, path):
    df['country'] = 'United States'
    df.to_csv(path)

In [707]:
zip_to_cartodb(ssbyzcta, 'data/for_cartodb/ss_by_zcta.csv')
zip_to_cartodb(ss_orig_byzcta, 'data/for_cartodb/ssorig_by_zcta.csv')

In [711]:
zip_to_cartodb(ss_by_ct, 'data/for_cartodb/ss_by_tract.csv')
zip_to_cartodb(ssorig_by_ct, 'data/for_cartodb/ssorig_by_tract.csv')

#### Actual crime scores

In [388]:
def get_crimes_by_zip(df):
    """Given original crimes df, return df of counts by crime type by zip code."""
    byzip = df.groupby(['zcta', 'Offense'])
    crimesdf = byzip.OBJECTID.count().unstack(level=-1)
    crimesdf.fillna(0, inplace = True)
    crimesdf['total'] = crimesdf.sum(axis = 1)
    crimesdf['zcta'] = crimesdf.index
    return crimesdf

In [362]:
def clean_acs(df, zcta_col, data_col):
    """Given ACS zcta data file, return dictionary with key = zcta and values = data."""
    df['zcta'] = df[zcta_col].apply(lambda x: int(x.split()[1]) if len(x.split()) > 1 else x)
    data_dict = dict(zip(df.zcta.values, df[data_col].values.tolist()))
    return data_dict

In [363]:
def clean_acs_df(df, zcta_col, data_col):
    """Given ACS zcta data file, return df of relevant."""
    df['zcta'] = df[zcta_col].apply(lambda x: int(x.split()[1]) if len(x.split()) > 1 else x)
    keep = data_col + ['zcta']
    return df[keep]

In [ ]:
"""normalizing crime:
- crime rate - crime per 100,000 residents
- great post: http://opendata.stackexchange.com/questions/381/how-to-normalize-the-data-when-mapping-crime-reports
"""

In [389]:
crbyzip2014 = get_crimes_by_zip(crime2014)

Normalize data by adding crime rates.

In [318]:
pop = pd.read_csv('data/acs-pop-by-zcta.csv')

# create dictionary of key = zip code, and value = population
pop_zcta = clean_acs(pop, 'GEO.display-label', ['HC01_VC03'])

In [391]:
# map population and divide crime by population
crbyzip2014['population'] = crbyzip2014.zcta.apply(lambda x: int(pop_zcta[x][0]) if x in pop_zcta else np.nan)
crbyzip2014['crime_p_1k'] = (crbyzip2014.total / crbyzip2014.population) * 1000

In [725]:
# drop nans and 0s
crbyzip2014.dropna(subset = ['crime_p_1k'], inplace = True)
crbyzip2014 = crbyzip2014[crbyzip2014.population > 1]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [729]:
zip_to_cartodb(crbyzip2014, 'data/for_cartodb/crime_by_zip_2014.csv')

#### Compare crime vs. perceived safety

Potential scores:
* ratio of perceived safety (or perceived crime, inverse) to actual crime rate
* quantile/rank of zip compared to other zip codes? e.g. 5th most "least safe" (dangerous) vs. 100th highest crime rate

In [736]:
ssbyz_danger.head()

,q_norm_rev
zcta,
10001,3.619298
10002,3.829057
10003,3.341040
10004,4.566766
10005,4.159614


In [737]:
crbyzip2014.head()

Offense,BURGLARY,FELONY ASSAULT,GRAND LARCENY,GRAND LARCENY OF MOTOR VEHICLE,MURDER & NON-NEGL. MANSLAUGHTE,RAPE,ROBBERY,total,zcta,population,crime_p1000,crime_p_1k,country
zcta,,,,,,,,,,,,,
10001,134,40,836,16,0,0,52,1078,10001,22767,4734.923354,47.349234,United States
10002,88,190,566,35,1,6,143,1029,10002,79894,1287.956542,12.879565,United States
10003,97,69,862,24,0,19,70,1141,10003,57068,1999.369174,19.993692,United States
10004,22,12,92,2,0,0,4,132,10004,3024,4365.079365,43.650794,United States
10005,4,7,114,6,0,0,11,142,10005,7570,1875.825627,18.758256,United States


In [738]:
ratios_by_zip = ssbyz_danger.merge(crbyzip2014, left_index = True, right_index = True)

In [740]:
keep = ['q_norm_rev', 'crime_p_1k']

In [741]:
ratios = ratios_by_zip[keep]

In [743]:
ratios['crime_ratio'] = ratios.q_norm_rev / ratios.crime_p_1k

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [745]:
ratios.sort_values(by = 'crime_ratio', ascending = False)

,q_norm_rev,crime_p_1k,crime_ratio
zcta,,,
10280,4.732504,1.814676,2.607906
10312,3.737261,2.798462,1.335470
10314,3.939777,3.713267,1.061000
11357,3.682912,3.523209,1.045329
11363,4.590532,4.459016,1.029494
10308,3.918249,4.261030,0.919554
11430,4.997403,5.847953,0.854556
10307,3.704024,4.398241,0.842160
10305,4.120639,5.039666,0.817641


### Test functions

In [61]:
testlist = zip_list[:5]
testpoint = zip_list[0][0]

In [74]:
tp2 = (40.6608, -73.9515)

In [73]:
get_zip(testpoint, testlist)

10001

In [ ]:
40.8115	-73.9668

In [171]:
get_zip_diff((40.7719, -73.9931), zip_list)

11371

In [62]:
print testlist
print testpoint

[((40.750741999999995, -73.996530000000007), 10001), ((40.717040000000004, -73.986999999999995), 10002), ((40.732509, -73.989350000000002), 10003), ((40.706018999999998, -74.008580000000009), 10005), ((40.707903999999999, -74.013419999999996), 10006)]
(40.750741999999995, -73.996530000000007)


In [194]:
# test some more
with open('data/ss_orig_withzips.pkl', 'r') as picklefile:
    ssorig = pickle.load(picklefile)

In [136]:
%time add_zips_df(ssorig)

CPU times: user 11.1 s, sys: 83.9 ms, total: 11.2 s
Wall time: 11.2 s


In [201]:
ssorig.zipcode2 = ssorig.zipcode2.apply(str)
ssorig.zipcode_diff = ssorig.zipcode_diff.apply(str)

In [202]:
ssorig['zip_error_dist'] = ssorig.zipcode == ssorig.zipcode2
ssorig['zip_error_diff'] = ssorig.zipcode == ssorig.zipcode_diff

In [205]:
# 25% incorrect... 
print ssorig.zip_error_dist.value_counts()[0]/float(ssorig.shape[0])
print ssorig.zip_error_diff.value_counts()[0]/float(ssorig.shape[0])

0.256364712848
0.269390171699


In [195]:
add_zips_df_diff(ssorig, 'Lat', 'Lon')

In [199]:
add_zips_df(ssorig, 'Lat', 'Lon')

In [203]:
ssorig

,City,Error in QS Safer,Error in QS Unique,Error in QS Upperclass,File_Location,Heading,ID,Lat,Lon,Pitch,QS Safer,QS Unique,QS Upperclass,zipcode,fips,zipcode_diff,zipcode2,zip_error_dist,zip_error_diff
1226,New York City,0.47,0.58,0.56,/images/id_825_400_300.jpg,123,825,40.6608,-73.9515,4,4.66,4.67,4.62,11225,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11225,11225,True,True
1227,New York City,0.44,0.73,0.55,/images/id_824_400_300.jpg,194,824,40.7667,-73.9385,7,3.98,4.98,4.98,11106,"{u'County': {u'FIPS': u'36081', u'name': u'Que...",11106,11106,True,True
1228,New York City,0.45,0.68,0.54,/images/id_827_400_300.jpg,184,827,40.8115,-73.9668,2,4.81,4.55,4.28,10027,"{u'County': {u'FIPS': u'36061', u'name': u'New...",10115,10115,False,False
1229,New York City,0.35,0.51,0.45,/images/id_826_400_300.jpg,156,826,40.7719,-73.9931,3,3.03,4.37,4.33,10019,"{u'County': {u'FIPS': u'36061', u'name': u'New...",10069,10069,False,False
1230,New York City,0.49,0.71,0.59,/images/id_821_400_300.jpg,166,821,40.7635,-73.8676,4,5.46,4.28,5.06,11369,"{u'County': {u'FIPS': u'36081', u'name': u'Que...",11369,11369,True,True
1232,New York City,0.54,0.74,0.51,/images/id_823_400_300.jpg,152,823,40.7541,-73.9902,5,5.23,5.41,4.93,10018,"{u'County': {u'FIPS': u'36061', u'name': u'New...",10123,10018,True,False
1233,New York City,0.31,0.65,0.14,/images/id_822_400_300.jpg,200,822,40.7266,-73.9390,4,3.21,4.17,3.22,11222,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11222,11222,True,True
1236,New York City,0.49,0.77,0.58,/images/id_4258_400_300.jpg,349,4258,40.6128,-74.0011,17,4.35,4.18,4.03,11214,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11228,11228,False,False
1238,New York City,0.57,0.86,0.77,/images/id_4312_400_300.jpg,20,4312,40.6438,-73.9668,3,6.86,5.15,7.78,11218,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11218,11218,True,True
1256,New York City,0.44,0.50,0.56,/images/id_797_400_300.jpg,167,797,40.6430,-74.0114,1,6.12,6.45,6.30,11220,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11220,11220,True,True


In [434]:
ssorig.loc[1226, 'fips']

{u'Block': {u'FIPS': u'360470800002000'},
 u'County': {u'FIPS': u'36047', u'name': u'Kings'},
 u'State': {u'FIPS': u'36', u'code': u'NY', u'name': u'New York'},
 u'executionTime': u'192',
 u'status': u'OK'}

In [425]:
ss.head()

,latitude,longitude,q-score,zipcode,q_norm,zcta
0,40.700909,-74.013504,11.062166,10041,3.204716,10004
1,40.701000,-74.013878,10.818611,10041,3.152896,10004
2,40.701080,-74.012878,12.677955,10041,3.548501,10004
3,40.701187,-74.013268,11.417325,10041,3.280282,10004
4,40.701244,-74.012115,25.199091,10041,6.212573,10004
